Extrator de Estruturas
=============
Extrai arestas a partir de um dado sísmico

---

In [1]:
from __future__ import print_function
import numpy as np
from skimage.morphology import skeletonize

# Importa bibliotecas próprias
from utils import * 

# Configura o matplolib para plotar inline
%matplotlib qt5
#%matplotlib notebook

In [2]:
# Importa dado

# Definição de arquivos de entrada
stack_file = './reflexoes_migradas/solimoes_migrado.sgy'

# Definição plot
color = 'gray'
clip = 0.3

# Carrega dados
if not stack_file is None: 
    traces = load_traces(stack_file)
    stack_image = get_data_from_traces(traces)
    print("Dimensões do dado 1:", stack_image.shape)
    print()
    
multiple_event_viewer(stack_image, cdp_offset=traces[0].cdp, dt=traces[0].dt,
             clip_percent1=clip, color1=color)

Carregando o empilhamento ./reflexoes_migradas/solimoes_migrado.sgy
2206 traços:
 - CDP: [17 - 2223]
 - Amostras por traço (ns): 751
 - Tempo entre amostras (dt): 4.0 ms
Dimensões do dado 1: (751, 2206)

Tamanho da imagem (em polegadas): 22.06 x 7.51
Coordenadas:  x - [17.0, 2223.0], y - [0.0, 3.0]
Dimensão: (751, 2206)



In [ ]:
# Extrator de arestas

# Definição filtro
threshold = 0.12

# Definição plot
color = 'gray'
clip = 0.3

# Normaliza
vmax = np.abs((stack_image.min(), stack_image.max())).max()
stack_image /= vmax
    
# Torna dado binario
binary_image = stack_image.copy()
indices = np.where(binary_image < 0.0)
binary_image[indices] = 0.0
binary_image /= binary_image.max()
binary_image[binary_image < threshold] = 0.0
binary_image[binary_image >= threshold] = 1.0

# Aplica detector de arestas
binary_edges = skeletonize(binary_image)
gray_edges = binary_edges * stack_image

multiple_event_viewer(stack_image, gray_edges,
             vmin2=gray_edges.min(), vmax2=gray_edges.max(),
             cdp_offset=traces[0].cdp, dt=traces[0].dt,
             clip_percent1=clip, color1=color, color2=color)

In [37]:
def agc(stack_image, window_size, dt):
    window_size = int(np.floor(window_size/dt))
    agc_image = stack_image.copy()
    abs_image = np.abs(stack_image)
    vmax = abs_image.max()
    end = stack_image.shape[0]-window_size
    for i in range(end):
        means = np.mean(abs_image[i:i+window_size,:], axis=0)
        correct_factor = vmax / means
        agc_image[i,:] = stack_image[i,:] * correct_factor
    # Trata o final do dado
    #for i in range(stack_image.shape[0],end-1,-1):
    #    means = np.mean(abs_image[i-window_size:i,:], axis=0)
    #    agc_image[i-1,:] = stack_image[i-1,:] / means
    for i in range(end,stack_image.shape[0]):
        agc_image[i,:] = stack_image[i,:] / means
    
    return agc_image

In [38]:
agc_image = agc(stack_image, 0.1, traces[0].dt)

# Normaliza
vmax = np.abs(agc_image).max()
agc_image /= vmax

In [39]:
# Definição plot
color = 'seismic'
clip = 1.0

    
multiple_event_viewer(agc_image, cdp_offset=traces[0].cdp, dt=traces[0].dt,
             clip_percent1=clip, color1=color)

Tamanho da imagem (em polegadas): 22.06 x 7.51
Coordenadas:  x - [17.0, 2223.0], y - [0.0, 3.0]
Dimensão: (751, 2206)

evento=1.0, cdp=1070.155836, tempo=2.920288


In [45]:
# Extrator de arestas

# Definição filtro
threshold = 0.02

# Definição plot
color = 'seismic'
clip = 0.5

# Normaliza
vmax = np.abs((stack_image.min(), stack_image.max())).max()
stack_image /= vmax
    
# Torna dado binario
binary_image = agc_image.copy()
indices = np.where(binary_image < 0.0)
binary_image[indices] = 0.0
binary_image /= binary_image.max()
binary_image[binary_image < threshold] = 0.0
binary_image[binary_image >= threshold] = 1.0

# Aplica detector de arestas
binary_edges = skeletonize(binary_image)
gray_edges = binary_edges * stack_image

multiple_event_viewer(stack_image, gray_edges,
             vmin2=gray_edges.min(), vmax2=gray_edges.max(),
             cdp_offset=traces[0].cdp, dt=traces[0].dt,
             clip_percent1=clip, color1=color, color2='gray')

Tamanho da imagem (em polegadas): 22.06 x 7.51
Coordenadas:  x - [17.0, 2223.0], y - [0.0, 3.0]
Dimensão: (751, 2206)

evento=1.0, cdp=373.387381, tempo=0.610112
evento=1.0, cdp=483.951348, tempo=1.122960
evento=1.0, cdp=1775.873467, tempo=0.709988
evento=1.0, cdp=2085.812807, tempo=1.050762
evento=1.0, cdp=1909.985060, tempo=1.391219
evento=1.0, cdp=1748.248038, tempo=1.498365
evento=1.0, cdp=1864.037042, tempo=1.568105
evento=1.0, cdp=1857.298000, tempo=1.706950
evento=1.0, cdp=1889.767932, tempo=1.797612
evento=1.0, cdp=1886.704731, tempo=2.042336
evento=1.0, cdp=1842.594634, tempo=2.264869
evento=1.0, cdp=1840.144073, tempo=2.436682
evento=1.0, cdp=1807.061501, tempo=2.605960
evento=1.0, cdp=1944.905553, tempo=2.800597


In [41]:
# Definição plot
color = 'seismic'
clip = 1.0

vmax = np.abs(stack_image).max()
stack_image /= vmax
    
multiple_event_viewer(agc_image, stack_image, cdp_offset=traces[0].cdp, dt=traces[0].dt,
             clip_percent1=clip, color1=color, clip_percent2=clip, color2=color)

Tamanho da imagem (em polegadas): 22.06 x 7.51
Coordenadas:  x - [17.0, 2223.0], y - [0.0, 3.0]
Dimensão: (751, 2206)

evento=1.0, cdp=1592.706365, tempo=0.531948
evento=1.0, cdp=1809.114152, tempo=0.890951
evento=1.0, cdp=1884.808858, tempo=1.036702
evento=1.0, cdp=2019.855097, tempo=1.136780
evento=1.0, cdp=1900.291867, tempo=1.079688
evento=1.0, cdp=898.995160, tempo=1.387408
evento=1.0, cdp=1084.005489, tempo=1.732677
evento=1.0, cdp=1433.157457, tempo=1.452544
evento=1.0, cdp=1463.017497, tempo=1.508873


In [46]:
# Salva dados
save_file = 'estruturas/solimoes_edges.pickle'

pickle_data(save_file, [gray_edges], ['edges'])

Dado persistido em estruturas/solimoes_edges.pickle
